准备数据

In [3]:
from numpy import *
import matplotlib.pyplot as plt

def loadDataSet(fileName):
    """
    Desc:
    """
    dataMat = []
    labelMat = []
    with open(fileName, 'r') as fr:
        for line in fr.readlines():
            lineArr = line.strip().split('\t')
            dataMat.append([float(lineArr[0]), float(lineArr[1])])
            labelMat.append(float(lineArr[2]))
        return dataMat, labelMat

In [4]:
def selectJrand(i, m):
    """
    随机选择一个整数
    Args:
      i 第一个alpha的下标
      m 所有alpha的目标
    Returns；
      j 返回一个不为i的随机数,在0-m之间的整数值
    """
    j = i
    while j == i:
        j = int(random.uniform(0, m))
    return j

def clipAlpha(aj, H, L):
    """
    Args:
      aj 目标值
      H  最大值
      L  最小值
    Returns:
      aj  目标值
    """
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

5


*** 训练算法 ***

In [ ]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    """
    Desc:
      smoSimple
    Args:
       dataMatIn  特征集合
       classLabels 类别标签
       C 松弛变量 允许有些数据点可以处于分隔面的错误一侧
         控制最大化间隔和保证大部分函数的间隔小于1.0这两个目标的权重
       toler 容错率(是指在某个体系中能减小一些因素或选择对某个系统产生不稳定的概率)
       maxIter 退出最大循环的次数
    Returns:
       b 模型的常量
       alphas 拉格朗日乘子
    """
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLables).transpose()
    m, n = shape(dataMatrix)
    
    # 初始化b和alphas(类似于权重分析)
    b = 0
    alphas = mat(zeros((m, 1)))
    
    # 没有任何alpha改变的情况下遍历数据的次数
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fxi = float(multiply(alphas, labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fxi - float(labelMat[i])
            
            if (((labelMat[i]*Ei <-toler) and (alphas[i] < C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)):
                # 如果满足优化条件就随机选取非i的一个节点
                j = selectJrand(i, m)
                # 预测j的结果
                fxj = float(multiply(alphas, labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fxj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                #
                if (labelMat[i] != labelMat[j]):
                   L = max(0, alphas[j] - alphas[i])
                   H = min(C, C+alphas[j] - alphas[i])
                else:
                   L = max(0, alphas[j] + alphas[i] - C)
                   H = min(C, alphas[j] + alphas[i])
                if L == H:
                  print ("L==H")
                  continue
                
                #eta是alphas[j]
                eta = 2.0 * dataMatrix[i, :]*dataMatrix[j, :].T - dataMatrix[i, :]*dataMatrix[i, :].T - dataMatrix[j, :]*dataMatrix[j, :].T
                if eta >= 0:
                    print("eta>=0")
                    continue
                # 计算出一个新的alphas[j]值
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                # 并使用辅助函数，以及L和H对其进行调整
                alphas[j] = clipAlpha(alphas[j], H, L)
                # 检查alpha[j]是否只是轻微的改变，如果是的话，就退出for循环。
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue
                # 然后alphas[i]和alphas[j]同样进行改变，虽然改变的大小一样，但是改变的方向正好相反
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                # 在对alpha[i], alpha[j] 进行优化之后，给这两个alpha值设置一个常数b。
                # w= Σ[1~n] ai*yi*xi => b = yj- Σ[1~n] ai*yi(xi*xj)
                # 所以：  b1 - b = (y1-y) - Σ[1~n] yi*(a1-a)*(xi*x1)
                # 为什么减2遍？ 因为是 减去Σ[1~n]，正好2个变量i和j，所以减2遍
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[i, :].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i, :]*dataMatrix[j, :].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[j, :].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j, :]*dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
  # 在for循环外，检查alpha值是否做了更新，如果更新则将iter设为0后继续运行程序
        # 直到更新完毕后，iter次循环无变化，才退出循环。
        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)
    return b, alphas              